In [185]:
#Read input graph and data
def read_initial_node():
    global initial_node 
    initial_node = int(input('Enter Start node : '))

def read_goal_node():
    global goal_node
    goal_node = int(input('Enter goal node : '))
    
def read_number_of_nodes():
    global nodes_count
    nodes_count = int(input('Enter number of nodes : '))

def read_graph():
    global transition_matrix
    transition_matrix = np.matrix(np.ones((nodes_count,nodes_count))*np.inf)
    for each_node in range(nodes_count):
        transition_matrix[each_node,each_node]=0
        edges_with_cost = input('Enter edges from node {0} : '.format(each_node)).split(',')
        for each_edge in edges_with_cost:
            if each_edge:
                transition_matrix[each_node,int(each_edge.split(':')[0])]=int(each_edge.split(':')[1])
                transition_matrix[int(each_edge.split(':')[0]),each_node]=int(each_edge.split(':')[1])

In [7]:
import numpy as np

def read_data():
    read_number_of_nodes()
    read_graph()

read_data()

Enter number of nodes : 8
Enter edges from node 0 : 1:15,2:10,3:17,6:5
Enter edges from node 1 : 3:12
Enter edges from node 2 : 6:7
Enter edges from node 3 : 4:2,5:10,7:4
Enter edges from node 4 : 
Enter edges from node 5 : 7:11
Enter edges from node 6 : 7:25
Enter edges from node 7 : 


In [186]:
def node_reach_goal_node(path):
    if path[-1] == goal_node:
        return True
    return False
def get_adj_nodes(nodes):
    adj_nodes = []
    for adj_node in range(nodes_count):
        if transition_matrix[nodes[-1],adj_node] not in (0,float('inf')) and adj_node not in nodes:
            adj_nodes.append(nodes + [adj_node])
    return adj_nodes

In [187]:
def find_cost(path):
    cost=0
    for index in range(len(path)-1):
        cost += transition_matrix[path[index],path[index+1]]
    return cost
def display_path(path):
    delim = "->"
    print('path :',str(delim.join(map(str, path))))

In [188]:
def dfs(dfs_stack=[[initial_node]]):
    if node_reach_goal_node(dfs_stack[-1]):
        return dfs_stack[-1]
    else:
        nodes = dfs_stack.pop()
        adj_nodes = get_adj_nodes(nodes)
        if len(adj_nodes) > 0:
            dfs_stack += adj_nodes
        return dfs(dfs_stack)

In [189]:
def all_possible_paths(dfs_stack=[[initial_node]],paths=[]):
    if len(dfs_stack) == 0:
        total_cost = 0
        for nodes in paths:
            total_cost += find_cost(nodes)
        avg_cost = total_cost/len(paths)
        return avg_cost
    elif node_reach_goal_node(dfs_stack[-1]):
        print(dfs_stack[-1])
        paths.append(dfs_stack[-1])
        dfs_stack.pop()
        return all_possible_paths(dfs_stack,paths)
    else:
        nodes = dfs_stack.pop()
        adj_nodes = get_adj_nodes(nodes)
        if len(adj_nodes) > 0:
            dfs_stack += adj_nodes
        return all_possible_paths(dfs_stack,paths)

In [190]:
read_initial_node()
read_goal_node()
display_pathsdfs())

Enter Start node : 0
Enter goal node : 5
[0, 6, 7, 5]
